In [9]:
import import_ipynb
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utilities.UtilityFunctions import retrieve_reviews_df, preprocessing, vectorize, text_preprocess

from statsmodels.miscmodels.ordinal_model import OrderedModel

In [13]:
df = preprocessing(retrieve_reviews_df())

Concatenating ../data\Reviews-1.csv
Concatenating ../data\Reviews-2.csv
Concatenating ../data\Reviews-3.csv
Concatenating ../data\Reviews-4.csv


In [14]:
# perform text preprocessing
text_preprocess(df, ['html', 'punc', 'lower'])

In [27]:
X = df[['Clean_text']]
y = df['Sentiment']

In [28]:
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=[-1, 0, 1], ordered=True)
y = y.astype(cat_type)

In [29]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)
X_train_counts, X_train_tfidf = vectorize(X_train)

In [30]:
mod_prob = OrderedModel(y_train, X_train, distr='probit')
res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [31]:
np.asarray(X_train)

array([['tasty by itself but not much more sugar than regular peanut butter this stuff is delicious especially for those who love the salty sweetness of chocolate n peanut butter ive used this as if it were a sort of fudge for ice cream heat up one serving for about 30 seconds in the microwave and then throw the ice cream in and enjoy very yummy'],
       ['john wm macys crunchy snack sticks are great  i have loved john wm macys sweet sticks for years the chocolate sticks are my favorite along with the java cinnamon sticks if you like a light slightly sweet snack or some crunchy nibbles with your coffee you cant go wrong  my suggestion is to order in bulk the chocolate ones are not offered on amazon yet and the cheddar stick and snacks  are good as well  but i love the lightly sweetened cinnamon and chocolate ones better  they are great for snacking or with coffee and breakfast  most of all  i love the products because of the ingredients  im intollerant to margarines and palm oil produ

In [25]:
df[['Clean_text']]

,Clean_text
0,i have bought several of the vitality canned d...
1,product arrived labeled as jumbo salted peanut...
2,this is a confection that has been around a fe...
3,if you are looking for the secret ingredient i...
4,great taffy at a great price there was a wide...
...,...
568449,great for sesame chickenthis is a good if not ...
568450,im disappointed with the flavor the chocolate ...
568451,these stars are small so you can give 1015 of ...
568452,these are the best treats for training and rew...


In [26]:
[X_train]

[321645    tasty by itself but not much more sugar than r...
 72131     john wm macys crunchy snack sticks are great  ...
 80123     i have used this product for a long time as li...
 49098     i cant find this flavor in my local grocery an...
 317894    no apple flavor what so ever has a horrible me...
                                 ...                        
 110272    yes this is spendy for genmai cha the thing is...
 259188    these are great the texture is like no other f...
 365856    we give these several times a week to all our ...
 131937    but thankfully amazon is there to be my suppli...
 121963    we have four dogs and they have done very well...
 Name: Clean_text, Length: 454741, dtype: object]